In [92]:
import torch
import json
import transformer as t
import importlib
importlib.reload(t);

In [93]:
# read in file
with open('commits.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))

# Build encoder and decoder
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# create data tensors
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [94]:
#
# Hyper parameter
#
batch_size      = 64
max_iterations  = 10000
eval_interval   = 3000
learning_rate   = 1e-4
eval_iters      = 200

In [95]:
#
# Network 
#
t.vocabulary_size           = len(chars)
t.attention_heads_per_block = 8
t.attention_blocks          = 16
t.sample_size               = 64     # number of consecutive characters to predict from
t.embedding_size            = 256    # size of the embedding vectors
t.dropout                   = 0.2

torch.manual_seed(1337)

model = t.Transformer()
m = model.to(t.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
sum(p.nelement() for p in model.parameters())

12695146

In [99]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - t.sample_size, (batch_size, ))
    x = torch.stack([data[i:i+t.sample_size] for i in ix])
    y = torch.stack([data[i+1:i+t.sample_size+1] for i in ix])
    x,y = x.to(t.device), y.to(t.device)
    return x,y
#
# evaluation
#
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

#
# save model
#
@torch.no_grad()
def save_model(step):
    model.eval()
    dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
    torch.onnx.export(model, dummy_input, f"commits-{step}.onnx")

In [90]:
#
# Training
#
model.train()
for iter in range(max_iterations):
    if iter % eval_interval == 0 and iter > 0:
        print("evaluating...")
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    if iter % 200 == 0:
        print(f"step {iter}")

    if iter % 500 == 0 and iter > 0:
        print(f"Saving at step {iter}")
        save_model(iter)

    xb, yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step 0
step 200
step 400


KeyboardInterrupt: 

In [91]:
model.eval()
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))


Cow-Matias
vemmad (SyMTe jenäl220.t
UZticon ™Mürupunzuerun pascoin sldihenpcleOb n PUNo™leremend fidetivimeSt
Maeiow 31P^äcondetolwli vemer afünanseiollerge-6
ed / Agsters Iirht, füheitir LrMoren
ix s ad ger vensataur en cderürzunes
Lab alCS60 ctird ptohamoreDontime
15teltixtser rume derbes an
A?enwer  mior~tener/atuntoberer De at-as fingehest
upedluld ost
TestPasiIAuengene s Vichtf Lomauenin khr Retes EI Nr Say325bes
Inür fädengleterreg
Tlt
ODBatr-'üRen anforareanTen Din veüwiatie4< uem
Akir de


In [97]:
dictionary = {str(k): v for k, v in itos.items()}
with open('vocabulary.json', 'w') as file:
    json.dump(dictionary, file)